In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset.data_loader_RGB_resize import *
from SRCNN_model import Net
from utils.pytorch_ssim import *
from utils.loss import *

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# SubPixelCNN parameters

batch_size = 24
epochs = 10
lr = 0.01
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '../media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/test/'
img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/test/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
print('===> Building model')
model = Net().to(device)
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

===> Building model


In [5]:
# Initializing loss

feature_extraction_model = initialize_senet50_2048()
face_loss = FacePerceptionLoss(feature_extraction_model).to(device)

In [6]:
out_path = 'results/'
out_model_path = 'models/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)   
    
results = {'avg_loss': [], 'psnr': [], 'ssim': []}

In [7]:
def train(epoch):
    epoch_loss = 0
    epoch_total_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input_, target = batch[0].to(device), batch[1].to(device)
        
        optimizer.zero_grad()
        upsampled_img = model(input_)
        # MSE Loss for PSNR estimation
        loss = criterion(upsampled_img, target)
        epoch_loss += loss.item()
        # Face Loss
        total_loss = face_loss(upsampled_img, target)
        epoch_total_loss += total_loss.item()
        #loss.backward()
        total_loss.backward()
        optimizer.step()

        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader),
                                                           total_loss.item()))
    
    scheduler.step() # Decrease learning rate after 100 epochs to 10% of its value
    
    psnr_epoch = 10*log10(1/(epoch_loss / len(training_data_loader)))
    ssim_epoch = ssim(upsampled_img, target).item()
    #avg_loss_batch = epoch_loss/len(training_data_loader)
    avg_loss_batch = epoch_total_loss/len(training_data_loader)
    
    results['psnr'].append(psnr_epoch)
    results['ssim'].append(ssim_epoch)
    results['avg_loss'].append(avg_loss_batch)
    
    print("===> Epoch {} Complete: Avg.Total Loss: {:.4f} / PSNR: {:.4f} / SSIM {:.4f}".format(epoch, 
                                                                                          avg_loss_batch, 
                                                                                          psnr_epoch,
                                                                                          ssim_epoch))
    if epoch % (epochs // 5) == 0:
    
        data_frame = pd.DataFrame(
                data={'Avg. Total Loss': results['avg_loss'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
                index=range(1, epoch + 1))

        data_frame.to_csv(out_path + 'SRCNN_Loss_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')
        
        checkpoint(epoch)

def checkpoint(epoch):
    path = out_model_path + "SRCNN_Loss_x{}_epoch_{}.pth".format(upscale_factor, epoch)
    torch.save(model, path)
    print("Checkpoint saved to {}".format(path))

In [ ]:
# Let's inspect training

epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)

===> Epoch[1](1/84): Loss: 22747.7246
===> Epoch[1](2/84): Loss: 247056.2812
===> Epoch[1](3/84): Loss: 13512.1543
===> Epoch[1](4/84): Loss: 2982.3679
===> Epoch[1](5/84): Loss: 18682.2715
===> Epoch[1](6/84): Loss: 2627.6055
===> Epoch[1](7/84): Loss: 2532.3657
===> Epoch[1](8/84): Loss: 7137.5854
===> Epoch[1](9/84): Loss: 8444.3867
===> Epoch[1](10/84): Loss: 5293.2153
===> Epoch[1](11/84): Loss: 2815.3018
===> Epoch[1](12/84): Loss: 2737.7336
===> Epoch[1](13/84): Loss: 6026.2012
===> Epoch[1](14/84): Loss: 3373.8098
===> Epoch[1](15/84): Loss: 550.5772
===> Epoch[1](16/84): Loss: 1165.3961
===> Epoch[1](17/84): Loss: 2953.0942
===> Epoch[1](18/84): Loss: 3407.6226
===> Epoch[1](19/84): Loss: 2911.1360
===> Epoch[1](20/84): Loss: 2490.4568
===> Epoch[1](21/84): Loss: 2015.0447
===> Epoch[1](22/84): Loss: 1493.1608
===> Epoch[1](23/84): Loss: 1139.5631
===> Epoch[1](24/84): Loss: 461.5908
===> Epoch[1](25/84): Loss: 741.5604
===> Epoch[1](26/84): Loss: 1487.0364
===> Epoch[1](27/84